# NGA-West2 Prediction

**Outcome**
- Proposed a time series regression based on recurrent neural networks(LSTM).
- The aim was to achieve a novel approach for estimating the PGA, and PGV for shallow crustal
earthquakes by seven earthquake’s main components.

# Prepare DS

In [0]:
import subprocess


data = 'https://www.dropbox.com/.../nga_west_v1.xlsx'
current_dir = '/tmp/nga-prj/'

subprocess.run(["rm", "-rf", current_dir])
subprocess.run(["wget", data, "-P", current_dir])

!ls /tmp/nga-prj/

nga_west_v1.xlsx


# Import Packages

In [0]:
!pip install -U -q xlrd
!pip install -U -q scikit-learn
!pip install -U -q openpyxl
!pip install -U -q keras

In [0]:
# OpenCV
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

# GraphViz
!apt-get -qq install -y graphviz && pip install -q pydot

# 7zip
!apt-get -qq install -y libarchive-dev && pip install -q -U libarchive

In [0]:
from collections import Counter
import datetime
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow import layers
from tensorflow.contrib.layers import xavier_initializer
from tensorflow.python.ops.init_ops import glorot_uniform_initializer
from tensorflow import layers
import time
import timeit
import os

plt.style.use('seaborn-ticks')
%matplotlib inline

# Load the data

In [0]:
data = pd.read_excel(os.path.join(current_dir + 'nga_west_v1.xlsx'))
data.head()

,Earthquake Magnitude,Dip (deg),Mechanism Based on Rake Angle,ClstD (km),FW/HW Indicator,Vs30 (m/s) selected for analysis,Idirectivity,PGA (g),PGV (cm/sec),PGD (cm),...,T8.500S,T9.000S,T9.500S,T10.000S,T11.000S,T12.000S,T13.000S,T14.000S,T15.000S,T20.000S
0,6.0,75.0,0,2.86,na,593.35,0,0.157020,10.05400,3.005200,...,0.001521,0.001368,0.001237,0.001123,0.000938,0.000794,0.000681,0.000591,0.000517,0.000295
1,6.0,75.0,0,2.92,na,551.82,0,0.046423,0.64978,0.035305,...,0.000021,0.000019,0.000017,0.000015,0.000012,0.000010,0.000009,0.000008,0.000007,0.000004
2,5.8,90.0,0,71.57,na,219.31,0,0.040961,2.79070,0.402240,...,0.000255,0.000226,0.000201,0.000180,0.000147,0.000122,0.000103,0.000088,0.000077,0.000044
3,5.0,90.0,0,34.98,na,213.44,0,0.018449,0.64551,0.056359,...,0.000031,0.000027,0.000024,0.000022,0.000018,0.000016,0.000013,0.000012,0.000010,0.000006
4,5.5,90.0,0,53.58,na,219.31,0,0.122180,6.56390,0.733060,...,0.000401,0.000348,0.000309,0.000276,0.000228,0.000193,0.000166,0.000144,0.000126,0.000072


In [0]:
features = [
    'Earthquake Magnitude', 
    'Dip (deg)', 
    'Mechanism Based on Rake Angle', 
    'ClstD (km)', 
    'FW/HW Indicator',
    'Vs30 (m/s) selected for analysis',
    'Idirectivity'
]
measures = [
    'PGA (g)', 'PGV (cm/sec)', 'PGD (cm)', 'T0.010S', 'T0.020S', 'T0.022S', 
    'T0.025S', 'T0.029S', 'T0.030S', 'T0.032S', 'T0.035S', 'T0.036S', 
    'T0.040S', 'T0.042S', 'T0.044S', 'T0.045S', 'T0.046S', 'T0.048S', 
    'T0.050S', 'T0.055S', 'T0.060S', 'T0.065S', 'T0.067S', 'T0.070S', 
    'T0.075S', 'T0.080S', 'T0.085S', 'T0.090S', 'T0.095S', 'T0.100S', 
    'T0.110S', 'T0.120S', 'T0.130S', 'T0.133S', 'T0.140S', 'T0.150S', 
    'T0.160S', 'T0.170S', 'T0.180S', 'T0.190S', 'T0.200S', 'T0.220S', 
    'T0.240S', 'T0.250S', 'T0.260S', 'T0.280S', 'T0.290S', 'T0.300S', 
    'T0.320S', 'T0.340S', 'T0.350S', 'T0.360S', 'T0.380S', 'T0.400S', 
    'T0.420S', 'T0.440S', 'T0.450S', 'T0.460S', 'T0.480S', 'T0.500S', 
    'T0.550S', 'T0.600S', 'T0.650S', 'T0.667S', 'T0.700S', 'T0.750S', 
    'T0.800S', 'T0.850S', 'T0.900S', 'T0.950S', 'T1.000S', 'T1.100S', 
    'T1.200S', 'T1.300S', 'T1.400S', 'T1.500S', 'T1.600S', 'T1.700S', 
    'T1.800S', 'T1.900S', 'T2.000S', 'T2.200S', 'T2.400S', 'T2.500S', 
    'T2.600S', 'T2.800S', 'T3.000S', 'T3.200S', 'T3.400S', 'T3.500S', 
    'T3.600S', 'T3.800S', 'T4.000S', 'T4.200S', 'T4.400S', 'T4.600S', 
    'T4.800S', 'T5.000S', 'T5.500S', 'T6.000S', 'T6.500S', 'T7.000S', 
    'T7.500S', 'T8.000S', 'T8.500S', 'T9.000S', 'T9.500S', 'T10.000S', 
    'T11.000S', 'T12.000S', 'T13.000S', 'T14.000S', 'T15.000S', 'T20.000S',
]

columns = features + measures
data = data[columns]

# Preprocessing v1

In [0]:
def cv2number(item):
    try:
        item = int(item)
    except:
        item = 0
    
    return item

In [0]:
indices = [list(data.loc[data[column].apply(cv2number) == -999].index) for column in columns]
indices = [ind for sub_ind in indices for ind in sub_ind]
indices = list(set(indices))
len(indices)

13076

In [0]:
new_data = data.drop(indices)

In [0]:
unique_fw_hw = list(new_data['FW/HW Indicator'].unique())
new_data['FW/HW Indicator'] = new_data['FW/HW Indicator'].apply(lambda i: unique_fw_hw.index(i))

new_data.head()

,Earthquake Magnitude,Dip (deg),Mechanism Based on Rake Angle,ClstD (km),FW/HW Indicator,Vs30 (m/s) selected for analysis,Idirectivity,PGA (g),PGV (cm/sec),PGD (cm),...,T8.500S,T9.000S,T9.500S,T10.000S,T11.000S,T12.000S,T13.000S,T14.000S,T15.000S,T20.000S
0,6.0,75.0,0,2.86,0,593.35,0,0.157020,10.05400,3.005200,...,0.001521,0.001368,0.001237,0.001123,0.000938,0.000794,0.000681,0.000591,0.000517,0.000295
1,6.0,75.0,0,2.92,0,551.82,0,0.046423,0.64978,0.035305,...,0.000021,0.000019,0.000017,0.000015,0.000012,0.000010,0.000009,0.000008,0.000007,0.000004
2,5.8,90.0,0,71.57,0,219.31,0,0.040961,2.79070,0.402240,...,0.000255,0.000226,0.000201,0.000180,0.000147,0.000122,0.000103,0.000088,0.000077,0.000044
3,5.0,90.0,0,34.98,0,213.44,0,0.018449,0.64551,0.056359,...,0.000031,0.000027,0.000024,0.000022,0.000018,0.000016,0.000013,0.000012,0.000010,0.000006
4,5.5,90.0,0,53.58,0,219.31,0,0.122180,6.56390,0.733060,...,0.000401,0.000348,0.000309,0.000276,0.000228,0.000193,0.000166,0.000144,0.000126,0.000072


In [0]:
new_data.to_excel(os.path.join(current_dir + 'nga_west_v2.xlsx'), encoding='utf-8')

# Load the sanitized data

In [0]:
data = pd.read_excel(os.path.join(current_dir + 'nga_west_v2.xlsx'))
data.head()

,Earthquake Magnitude,Dip (deg),Mechanism Based on Rake Angle,ClstD (km),FW/HW Indicator,Vs30 (m/s) selected for analysis,Idirectivity,PGA (g),PGV (cm/sec),PGD (cm),...,T8.500S,T9.000S,T9.500S,T10.000S,T11.000S,T12.000S,T13.000S,T14.000S,T15.000S,T20.000S
0,6.0,75.0,0,2.86,0,593.35,0,0.157020,10.05400,3.005200,...,0.001521,0.001368,0.001237,0.001123,0.000938,0.000794,0.000681,0.000591,0.000517,0.000295
1,6.0,75.0,0,2.92,0,551.82,0,0.046423,0.64978,0.035305,...,0.000021,0.000019,0.000017,0.000015,0.000012,0.000010,0.000009,0.000008,0.000007,0.000004
2,5.8,90.0,0,71.57,0,219.31,0,0.040961,2.79070,0.402240,...,0.000255,0.000226,0.000201,0.000180,0.000147,0.000122,0.000103,0.000088,0.000077,0.000044
3,5.0,90.0,0,34.98,0,213.44,0,0.018449,0.64551,0.056359,...,0.000031,0.000027,0.000024,0.000022,0.000018,0.000016,0.000013,0.000012,0.000010,0.000006
4,5.5,90.0,0,53.58,0,219.31,0,0.122180,6.56390,0.733060,...,0.000401,0.000348,0.000309,0.000276,0.000228,0.000193,0.000166,0.000144,0.000126,0.000072


In [0]:
features = [
    'Earthquake Magnitude', 
    'Dip (deg)', 
    'Mechanism Based on Rake Angle', 
    'ClstD (km)', 
    'FW/HW Indicator',
    'Vs30 (m/s) selected for analysis',
    # 'Idirectivity'
]
measures = [
    'PGA (g)', 'PGV (cm/sec)', 'PGD (cm)', 'T0.010S', 'T0.020S', 'T0.022S', 
    'T0.025S', 'T0.029S', 'T0.030S', 'T0.032S', 'T0.035S', 'T0.036S', 
    'T0.040S', 'T0.042S', 'T0.044S', 'T0.045S', 'T0.046S', 'T0.048S', 
    'T0.050S', 'T0.055S', 'T0.060S', 'T0.065S', 'T0.067S', 'T0.070S', 
    'T0.075S', 'T0.080S', 'T0.085S', 'T0.090S', 'T0.095S', 'T0.100S', 
    'T0.110S', 'T0.120S', 'T0.130S', 'T0.133S', 'T0.140S', 'T0.150S', 
    'T0.160S', 'T0.170S', 'T0.180S', 'T0.190S', 'T0.200S', 'T0.220S', 
    'T0.240S', 'T0.250S', 'T0.260S', 'T0.280S', 'T0.290S', 'T0.300S', 
    'T0.320S', 'T0.340S', 'T0.350S', 'T0.360S', 'T0.380S', 'T0.400S', 
    'T0.420S', 'T0.440S', 'T0.450S', 'T0.460S', 'T0.480S', 'T0.500S', 
    'T0.550S', 'T0.600S', 'T0.650S', 'T0.667S', 'T0.700S', 'T0.750S', 
    'T0.800S', 'T0.850S', 'T0.900S', 'T0.950S', 'T1.000S', 'T1.100S', 
    'T1.200S', 'T1.300S', 'T1.400S', 'T1.500S', 'T1.600S', 'T1.700S', 
    'T1.800S', 'T1.900S', 'T2.000S', 'T2.200S', 'T2.400S', 'T2.500S', 
    'T2.600S', 'T2.800S', 'T3.000S', 'T3.200S', 'T3.400S', 'T3.500S', 
    'T3.600S', 'T3.800S', 'T4.000S', 'T4.200S', 'T4.400S', 'T4.600S', 
    'T4.800S', 'T5.000S', 'T5.500S', 'T6.000S', 'T6.500S', 'T7.000S', 
    'T7.500S', 'T8.000S', 'T8.500S', 'T9.000S', 'T9.500S', 'T10.000S', 
    'T11.000S', 'T12.000S', 'T13.000S', 'T14.000S', 'T15.000S', 'T20.000S',
]

columns = features + measures

In [0]:
def cv2number(item):
    try:
        item = int(item)
    except:
        item = 0
    
    return item

indices = [list(data.loc[data[column].apply(cv2number) == -999].index) for column in columns]
indices = [ind for sub_ind in indices for ind in sub_ind]
indices = list(set(indices))
len(indices)

0

In [0]:
X = data[features]
Y = data[measures[0]]

In [0]:
X.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Earthquake Magnitude,8464.0,6.265542,0.871857,3.70,5.800,6.300,6.8000,7.90
Dip (deg),8464.0,56.365276,22.111235,10.00,36.000,50.000,78.0000,90.00
Mechanism Based on Rake Angle,8464.0,1.523511,1.184769,0.00,0.000,2.000,2.0000,4.00
ClstD (km),8464.0,121.120349,117.919620,0.05,43.705,91.875,171.3450,1532.66
FW/HW Indicator,8464.0,0.839319,0.657218,0.00,0.000,1.000,1.0000,3.00
Vs30 (m/s) selected for analysis,8464.0,410.456618,165.576299,89.32,300.220,376.960,498.1225,2100.00


In [0]:
Y.describe().transpose()

count    8464.000000
mean        0.064788
std         0.114333
min         0.000008
25%         0.008386
50%         0.023390
75%         0.071355
max         1.768300
Name: PGA (g), dtype: float64

# Preprocessing v2

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.02, random_state=42)
X_test, X_validation, y_test, y_validation = train_test_split(X_test, y_test, test_size=0.7, random_state=102)

print('Train Features Shape {} Lable Shape {}'.format(X_train.shape, y_train.shape))
print('Validation Features Shape {} Lable Shape {}'.format(X_validation.shape, y_validation.shape))
print('Test Features Shape {} Lable Shape {}'.format(X_test.shape, y_test.shape))

Train Features Shape (8294, 6) Lable Shape (8294,)
Validation Features Shape (119, 6) Lable Shape (119,)
Test Features Shape (51, 6) Lable Shape (51,)


In [0]:
y_train_data = y_train.as_matrix().reshape((-1, 1))
y_validation_data = y_validation.as_matrix().reshape((-1, 1))
y_test_data = y_test.as_matrix().reshape((-1, 1))

print(y_train_data.shape)
print(y_validation_data.shape)
print(y_test_data.shape)

(8294, 1)
(119, 1)
(51, 1)


In [0]:
X_train_data = X_train.as_matrix()
X_validation_data = X_validation.as_matrix()
X_test_data = X_test.as_matrix()

print(X_train_data.shape)
print(X_validation_data.shape)
print(X_test_data.shape)

(8294, 6)
(119, 6)
(51, 6)


# Implement the model

In [0]:
class DNNLayer(object):
    def __init__(self, units, activation=None, name=None, kernel_initializer=None, bias_initializer=None, **kwargs):
        self.units = units
        self.activation = activation

        self.name = name
        self.kernel_initializer = xavier_initializer() if kernel_initializer is None else kernel_initializer
        self.bias_initializer = xavier_initializer() if bias_initializer is None else bias_initializer
        self.use_bias = kwargs.get('use_bias', True)
        self.kernel_regularizer = kwargs.get('kernel_regularizer', None)
        self.bias_regularizer = kwargs.get('bias_regularizer', None)
        self.activity_regularizer = kwargs.get('activity_regularizer', None)
        self.kernel_constraint = kwargs.get('kernel_constraint', None)
        self.bias_constraint = kwargs.get('bias_constraint', None)
        self.trainable = kwargs.get('trainable', True)
        self.reuse = kwargs.get('reuse', None)

    def layer(self):
        layer = layers.Dense(units=self.units,
                             activation=self.activation,
                             use_bias=self.use_bias,
                             kernel_initializer=self.kernel_initializer,
                             bias_initializer=self.bias_initializer,
                             kernel_regularizer=self.kernel_regularizer,
                             bias_regularizer=self.bias_regularizer,
                             activity_regularizer=self.activity_regularizer,
                             kernel_constraint=self.kernel_constraint,
                             bias_constraint=self.bias_constraint,
                             trainable=self.trainable,
                             name=self.name,
                             _scope=self.name,
                             _reuse=self.reuse)

        return layer

    def build(self, inputs):
        return self.layer().apply(inputs)
    
    
class RNNR(object):
    verbose_indent = 35

    def __init__(self,
                 inputs_steps,
                 inputs_size,
                 outputs_steps,
                 outputs_size,
                 cell_units=100,
                 batch_size=1,
                 lr=1e-3,
                 lr_decay=.9,
                 lr_decay_step=100,
                 loss_type='',
                 optimizer_type='',
                 keep_prob=1.,
                 out_dir='runs',
                 checkpoint_dir='checkpoints',
                 num_checkpoints=5):

        self.inputs_size = inputs_size
        self.inputs_steps = inputs_steps
        self.outputs_size = outputs_size
        self.outputs_steps = outputs_steps

        self.cell_units = cell_units
        self.batch_size = batch_size

        self.out_dir = out_dir
        self.checkpoint_dir = checkpoint_dir
        self.lr = lr
        self.lr_decay = lr_decay
        self.lr_decay_step = lr_decay_step
        self.loss_type = loss_type
        self.optimizer_type = optimizer_type

        self.keep_prob = keep_prob
        self.num_checkpoints = num_checkpoints

        self.loss = None

    def set_session(self, session):
        self.session = session

    def arch(self):
        # Tensors where we will feed the data into graph
        self.inputs = tf.placeholder(tf.float32, shape=[None, self.inputs_steps, self.inputs_size], name='inputs')
        self.outputs = tf.placeholder(tf.float32, shape=[None, self.outputs_steps, self.outputs_size], name='outputs')
        self.outputs_r = tf.reshape(self.outputs, [-1, self.outputs_size], name='outputs_r')

        # Inputs operations
        self.inputs_r = tf.reshape(self.inputs, [-1, self.inputs_size], name='inputs_reshape')
        self.inputs_l = DNNLayer(
            self.cell_units,
            kernel_initializer=tf.random_normal_initializer(mean=0., stddev=1.),
            bias_initializer=tf.constant_initializer(.1),
            name='inputs_layer').build(self.inputs_r)
        self.inputs_l_r = tf.reshape(
            self.inputs_l, [-1, self.inputs_steps, self.cell_units], name='inputs_layer_reshape')

        # Cell operations
        self.cell = tf.nn.rnn_cell.LSTMCell(self.cell_units, name='cell')
        # self.cell = tf.nn.rnn_cell.GRUCell(self.cell_units, name='cell')
        self.cell = tf.nn.rnn_cell.DropoutWrapper(self.cell, input_keep_prob=self.keep_prob)
        self.cell_init_state = self.cell.zero_state(self.batch_size, dtype=tf.float32)
        self.cell_outputs, self.cell_final_state = tf.nn.dynamic_rnn(
            cell=self.cell,
            inputs=self.inputs_l_r,
            initial_state=self.cell_init_state)
        self.cell_outputs_r = tf.reshape(self.cell_outputs, [-1, self.cell_units], name='cell_outputs_r')

        self.logits = DNNLayer(
            units=self.outputs_size,
            name='logits').build(self.cell_outputs_r)

        self.losses, self.loss = self.compute_loss(self.logits)

        self.global_step = tf.Variable(0, name='global_step', trainable=False)
        self.lr = tf.train.exponential_decay(
            learning_rate=self.lr,
            global_step=self.global_step,
            decay_rate=self.lr_decay,
            decay_steps=self.lr_decay_step,
            staircase=True
        )

    def optimization(self, lr):
        if self.optimizer_type.lower() == 'adadelta':
            return tf.train.AdadeltaOptimizer(learning_rate=lr)

        elif self.optimizer_type.lower() == 'adam':
            return tf.train.AdamOptimizer(learning_rate=lr)

        elif self.optimizer_type.lower() == 'rmsprop':
            return tf.train.RMSPropOptimizer(learning_rate=lr)

        else:
            return tf.train.GradientDescentOptimizer(learning_rate=lr)

    @staticmethod
    def ms_error(labels, logits):
        # labels = tf.log(labels)
        # logits = tf.log(logits)
        return tf.square(tf.subtract(labels, logits))

    @staticmethod
    def mse_error(labels, logits):
        ms = tf.square(tf.subtract(labels, logits))
        return tf.reduce_mean(ms)

    @staticmethod
    def rmse_error(labels, logits):
        mse = tf.reduce_mean(tf.square(tf.subtract(labels, logits)))
        return tf.sqrt(mse)

    @staticmethod
    def error(loss_type=''):
        if loss_type.lower() == 'ms':
            return RNNR.ms_error
        elif loss_type.lower() == 'mse':
            return RNNR.mse_error
        elif loss_type.lower() == 'rmse':
            return RNNR.rmse_error
        else:
            return RNNR.ms_error
    
    def compute_loss(self, logits):
        loss_logits = tf.reshape(logits, [-1], name='loss_logits')
        loss_targets = tf.reshape(self.outputs, [-1], name='loss_logits')
        loss_weights = tf.ones([self.batch_size * self.outputs_size], name='loss_weights')

        losses = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            logits=[loss_logits],
            targets=[loss_targets],
            weights=[loss_weights],
            average_across_timesteps=True,
            softmax_loss_function=self.error(self.loss_type),
            name='losses')

        loss = tf.div(tf.reduce_sum(losses, name='losses_sum'), self.batch_size, name='average_loss')

        return losses, loss

    def operation(self):
        # Define Training procedure
        self.optimizer = self.optimization(lr=self.lr)
        self.grads_and_vars = self.optimizer.compute_gradients(self.loss)
        self.train_op = self.optimizer.apply_gradients(self.grads_and_vars, global_step=self.global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in self.grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram('{}/grad/hist'.format(v.name), g)
                sparsity_summary = tf.summary.scalar('{}/grad/sparsity'.format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        self.grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        self.out_dir = os.path.abspath(self.out_dir)
        print('Writing to {}\n'.format(self.out_dir))

        # Summaries for loss and accuracy
        self.loss_summary = tf.summary.scalar('loss', self.loss)

        self.train_summary_op = tf.summary.merge([self.loss_summary, self.grad_summaries_merged])
        self.validation_summary_op = tf.summary.merge([self.loss_summary])

        # Train Summaries
        self.train_summary_dir = os.path.join(self.out_dir, 'summaries', 'train')
        self.train_summary_writer = tf.summary.FileWriter(self.train_summary_dir, self.session.graph)

        # Test summaries
        self.validation_summary_dir = os.path.join(self.out_dir, 'summaries', 'validation')
        self.validation_summary_writer = tf.summary.FileWriter(self.validation_summary_dir, self.session.graph)

        # Checkpoint directory. TensorFlow assumes this directory already exists so we need to create it
        self.checkpoint_dir = os.path.abspath(os.path.join(self.out_dir, self.checkpoint_dir))
        self.checkpoint_prefix = os.path.join(self.checkpoint_dir, 'model')
        if not os.path.exists(self.checkpoint_dir):
            os.makedirs(self.checkpoint_dir)
        self.saver = tf.train.Saver(tf.global_variables(), max_to_keep=self.num_checkpoints)

        # Initialize all variables
        self.session.run(tf.global_variables_initializer())

    def build(self):
        print('Building the ANN...')
        self.arch()
        self.operation()

    def batching(self, X, Y=None, batch_size=None):
        batch_size = batch_size if batch_size else self.batch_size
        shuffle = np.random.permutation(len(X))
        start = 0
        X = X[shuffle]

        if Y is not None:
            Y = Y[shuffle]

            while start + batch_size <= len(X):
                yield X[start:start + batch_size], Y[start:start + batch_size]
                start += batch_size
        else:
            while start + batch_size <= len(X):
                yield X[start:start + batch_size]
                start += batch_size

    def training(self, inputs, outputs, start_time):
        feed_dict = {
            self.inputs: inputs,
            self.outputs: outputs,
        }

        _, step, summaries, loss = self.session.run([
            self.train_op, self.global_step, self.train_summary_op, self.loss], feed_dict)

        print('Epoch {:3}    Loss: {:>10.5f}    Epoch duration: {:>10.5f}s'.format(step, loss, time.time() - start_time))

        self.train_summary_writer.add_summary(summaries, step)

        return self.train_summary_writer

    def validating(self, inputs, outputs, writer=None):
        feed_dict = {
            self.inputs: inputs,
            self.outputs: outputs
        }

        _, step, summaries, loss = self.session.run([
            self.train_op, self.global_step, self.validation_summary_op, self.loss], feed_dict)

        print('Evaluation =>    Epoch {:3}    Loss: {:>10.5f}'.format(step, loss))

        if writer:
            writer.add_summary(summaries, step)

        return writer

    def fit(self, train_data, valid_data=None, epochs=1, evaluate_ey=10, checkpoint_ey=10, shuffling=False):
        X_tr, Y_tr = [None, None]
        X_val, Y_val = [None, None]
        validating = False
        train_summary_writer = None
        validation_summary_writer = None
        path = None

        if train_data is None or not isinstance(train_data, list):
            raise Exception('Your train data must be a list that consists of [X, Y]')
        elif not len(train_data) == 2:
            raise Exception('Your train data must be a list that consists of [X, Y]')
        else:
            X_tr, Y_tr = train_data

        if valid_data is not None:
            if valid_data is None or not isinstance(valid_data, list):
                raise Exception('Your validation data must be a list that consists of [X, Y]')
            elif not len(valid_data) == 2:
                raise Exception('Your validation data must be a list that consists of [X, Y]')
            else:
                X_val, Y_val = valid_data
                validating = True

        for epoch in range(epochs + 1):
            start_time = time.time()

            if shuffling:
                X_tr, Y_tr = shuffle(X_tr, Y_tr)

                if validating:
                    X_val, Y_val = shuffle(X_val, Y_val)

            for X_tr_batch, Y_tr_batch in self.batching(X_tr, Y_tr, self.batch_size):

                train_summary_writer = self.training(
                    inputs=X_tr_batch,
                    outputs=Y_tr_batch,
                    start_time=start_time
                )

                current_step = tf.train.global_step(self.session, self.global_step)

                if validating and current_step % evaluate_ey == 0:
                    print('\n')
                    print('-' * self.verbose_indent, 'Validation', '-' * self.verbose_indent)

                    for X_val_batch, Y_val_batch in self.batching(X_val, Y_val, self.batch_size):
                        validation_summary_writer = self.validating(
                            inputs=X_val_batch,
                            outputs=Y_val_batch,
                            writer=self.validation_summary_writer)

                    out = self.verbose_indent * 2 + 2 + len('Validation')
                    print('-' * out)
                    print('\n')

                if current_step % checkpoint_ey == 0:
                    path = self.saver.save(self.session, self.checkpoint_prefix, global_step=current_step)
                    print('\n')
                    print('-' * self.verbose_indent, 'Saved model checkpoint to', '-' * self.verbose_indent)
                    print(path)
                    out = self.verbose_indent * 2 + 2 + len('Saved model checkpoint to')
                    print('-' * out)
                    print('\n')

        # Fix for google-colab
        if train_summary_writer:
            train_summary_writer.flush()
            train_summary_writer.close()

        if validation_summary_writer:
            validation_summary_writer.flush()
            validation_summary_writer.close()
    
    
    def predicting(self, inputs, outputs, session, graph, checkpoint_dir=None):
        checkpoint_dir = checkpoint_dir if checkpoint_dir else self.checkpoint_dir
        checkpoint_file = tf.train.latest_checkpoint(checkpoint_dir)
        saver = tf.train.import_meta_graph('{}.meta'.format(checkpoint_file))
        saver.restore(session, checkpoint_file)

        operations = graph.get_operations()
        _inputs = graph.get_operation_by_name('inputs').outputs[0]
        # _outputs = graph.get_operation_by_name('outputs').outputs[0]
        _logits = graph.get_operation_by_name('logits/BiasAdd').outputs[0]

        
        
        feed_dict = {_inputs: inputs}
        logits = session.run(_logits, feed_dict=feed_dict)
        
        inputs = inputs.reshape((-1, inputs.shape[2]))
        outputs = outputs.reshape((-1, outputs.shape[2]))
        
        print('Average Loss: {:>10.6f}'.format(mean_squared_error(outputs, logits)))
        print()
        
        return inputs, outputs, logits, operations

In [1]:
out_dir = os.path.abspath(os.path.join(current_dir, 'outputs', str(int(time.time()))))

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    

model = RNNR(
    inputs_steps=1, inputs_size=X_train_data.shape[1],
    outputs_steps=1, outputs_size=y_train_data.shape[1],
    cell_units=300,
    batch_size=1,
    lr=1e-3,
    lr_decay=.8,
    lr_decay_step=100,
    loss_type='rmse',
    optimizer_type='adam',
    keep_prob=.5,
    num_checkpoints=5,
    out_dir=out_dir
)

train_data = [
    X_train_data.reshape((-1, 1, X_train_data.shape[1])), 
    y_train_data.reshape((-1, 1, y_train_data.shape[1]))]

valid_data = [
    X_validation_data.reshape((-1, 1, X_validation_data.shape[1])), 
    y_validation_data.reshape((-1, 1, y_validation_data.shape[1]))]

test_data = [
    X_test_data.reshape((-1, 1, X_test_data.shape[1])), 
    y_test_data.reshape((-1, 1, y_test_data.shape[1]))]


tf.reset_default_graph()

graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session = tf.Session(config=session_conf)
    
    with session.as_default():
        model.set_session(session)
        model.build()
        model.fit(
            train_data=train_data, valid_data=valid_data, epochs=1, shuffling=True, evaluate_ey=50, checkpoint_ey=50)

# Testing

In [0]:
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session = tf.Session(config=session_conf)
    
    with session.as_default():
        X, Y = next(model.batching(test_data[0], test_data[1]))
        
        _inputs, _outputs, _logits, _ = model.predicting(X, Y, session, graph)
        
#         checkpoint_dir = model.checkpoint_dir
#         checkpoint_file = tf.train.latest_checkpoint(checkpoint_dir)
#         saver = tf.train.import_meta_graph('{}.meta'.format(checkpoint_file))
#         saver.restore(session, checkpoint_file)

#         operations = graph.get_operations()
#         _inputs = graph.get_operation_by_name('inputs').outputs[0]
#         # _outputs = graph.get_operation_by_name('outputs').outputs[0]
#         _logits = graph.get_operation_by_name('logits/BiasAdd').outputs[0]

        
        
#         feed_dict = {_inputs: X}
#         logits = session.run(_logits, feed_dict=feed_dict)
        
#         inputs = inputs.reshape((-1, inputs.shape[2]))
#         outputs = outputs.reshape((-1, outputs.shape[2]))
        
#         print('Average Loss: {:>10.6f}'.format(mean_squared_error(outputs, logits)))
#         print()